In [2]:
# IMPORT REQUIRED LIBRARIES
# YOU SHOULD MAKE SURE THAT YOU ALREADY INSTLALED THEM PREVIOUSLY
import yfinance as yf
import finplot as fplt
# chamada_api = yf.Ticker('PETR4.SA').history(period='36mo')
# Create a finplot instance
chamada_api = yf.Ticker('PETR4.SA').history(period='36mo')
ax = fplt.create_plot(title='Gráfico de Velas', rows = 1, maximize=False)
# Plot the data on the graph
candles = chamada_api[['Open', 'Close', 'High', 'Low']]
daily_plot = fplt.candlestick_ochl(candles, candle_width=1)
daily_plot.colors.update(dict(bull_body='#bfb', bull_shadow='#ada', bear_body='#fbc', bear_shadow='#dab'))
# Add some additional features to the plot
fplt.show()